In [2]:
import pandas as pd
import re
import operator
import calendar
import urllib.request, urllib.error, urllib.parse
from bs4 import BeautifulSoup
# Get Tourneys
fileName=('BURecordBook.txt')
tourneys=[]
teamSet=set()
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    for i in rows:
        row=i.split('\t')
        if(len(row[0])>7 and len(row)==1 and ('COACH' not in i and 'OVERALL' not in i and 'ECAC:' not in i and 'CAPTAIN' not in i and 'HOCKEY' not in i and 'NEIHL:' not in i and 'forfeit' not in i)):
            tourneys.append(row[0])
            
            
tourneyDict={}
for i in tourneys:
    if(i=='Key to Tournaments'):
        continue
    tourney=i.split(' ')
    tourneyDict[tourney[0]]=' '.join(tourney[1:])
tourneyDict['nc']='Non-Colleigate'
tourneyDict['Oly nc']='1932 NEAAU Olympic tryouts-Non-Colleigate'
tourneyDict['ex']='Exhibition'
tourneyDict['HF ex']='Hall of Fame Game-Exhibition'
tourneyDict['IB ex']='Ice Breaker-Exhibition'

In [3]:
# Get Games
fileName=('BURecordBook.txt')
tourneys=[]
teamSet=set()
gameList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    conf='Independent'
    bean=0
    for i in rows:      
        if(len(i)==7):
            season=i
        if(len(row[0])>7 and len(row)==1 and ('COACH' not in i and 'OVERALL' not in i and 'ECAC:' not in i and 'CAPTAIN' not in i and 'HOCKEY' not in i and 'NEIHL:' not in i and 'forfeit' not in i)):
            tourneys.append(row[0])
            
        coachSearch=re.search("COACH: (.*)",i)
        if coachSearch != None:
            coach=coachSearch.group(1)
        captSearch=re.search("CAPTAIN.?: (.*)",i)
        if captSearch != None:
            capt=captSearch.group(1) 
            #print(capt)
        confSearch=re.search("(NEIHL|ECAC|HOCKEY EAST):",i)
        if confSearch != None:
            conf=confSearch.group(1)
            if(conf=='HOCKEY EAST'):
                conf='Hockey East'
        if(re.search('\*',i)!=None):
            gameType='Conference'
        else:
            gameType='Non-Conference'
        i=i.replace("* ",'')
        note=''
        if('†' in i):
            note='Loss by forfeit (ineligible player)'
            i=i.replace('†','')
        if('+' in i):
            note='Win by forfeit (opponent left ice – score was 5-1, BU)'
            i=i.replace('+','')
        if('‡' in i):
            note='Win by forfeit (ineligible player)'
            i=i.replace('‡','')
        game=re.search(r"(\d*\/\d*) (\w*) (?:\((.?ot)\))? ?(.*)\t(\S*|\S* \S*|\S* \S* \S*) ?(\(.*\))? (\d*-\d*)",i)
        if game==None:
            continue
        #print(game.groups())
        gameDict={'date':game.group(1),
                 'result':game.group(2),
                 'ot':game.group(3),
                 'arena':game.group(4),
                 'opponent':game.group(5),
                 'gameType':gameType,
                 'tourney': game.group(6),
                 'scoreline':game.group(7),
                 'location':'',
                 'coach':coach,
                 'captain':capt,
                 'conference':conf,
                 'season':season,
                 'note':note}
        if(gameDict['gameType']==None):
            gameDict['gameType']='Non-Conference'
        if(gameDict['tourney']=='(ex)' or gameDict['result']=='E'):
            gameDict['gameType']='Exhibition'
            gameDict['result'] = 'E'
            gameDict['tourney'] = None
        if(gameDict['arena']=='Agganis Arena' or gameDict['arena']=='Brown Arena' or gameDict['arena']=='Boston Arena'):
            gameDict['location']='Home'
        elif(gameDict['tourney']==None or gameDict['tourney']=='(nc)' or gameDict['tourney'] == '(B1G/HE)' or ((gameDict['tourney'] == '(HE)' or gameDict['tourney'] == '(ECAC)') and (gameDict['arena'] != 'TD Garden' and gameDict['arena'] != 'Boston Garden' and gameDict['arena'] != 'Providence CC'))):
            gameDict['location']='Away'
        if(gameDict['location']=='' or gameDict['arena']=='Boston Garden' or gameDict['arena']=='VW Arena'):
            gameDict['location']='Neutral'
        if((gameDict['arena']=='Gutterson' and gameDict['opponent']=='Vermont') or (gameDict['arena']=='Houston' and gameDict['opponent']=='Rensselaer') or (gameDict['arena']=='Broadmoor' and gameDict['opponent']=='Colorado College') or (gameDict['arena']=='DEC Center' and gameDict['opponent']=='Minnesota Duluth')or (gameDict['arena']=='Magness Arena' and gameDict['opponent']=='Denver')or (gameDict['arena']=='Mariucci Arena' and gameDict['opponent']=='Minnesota')or (gameDict['arena']=='Munn Ice Arena' and gameDict['opponent']=='Michigan State')or (gameDict['arena']=='Walker Arena' and gameDict['opponent']=='Clarkson')or (gameDict['arena']=='Thompson Arena' and gameDict['opponent']=='Dartmouth')or (gameDict['arena']=='St. Louis Arena' and gameDict['opponent']=='St. Louis') or (gameDict['arena']=='Sullivan Arena' and gameDict['opponent']=='Alaska Anchorage')):
            gameDict['location']='Away'

        if(gameDict['tourney']!=None):
            #print(gameDict)
            gameDict['tourney']=tourneyDict[gameDict['tourney'].replace('(','').replace(')','')]
        gameDict['month'],gameDict['day']=gameDict['date'].split('/')
        gameDict['month']=int(gameDict['month'])
        gameDict['day']=int(gameDict['day'])
        if(gameDict['month'] >=9):
            gameDict['date']+="/" + gameDict['season'][:4]
            gameDict['year']=int(gameDict['season'][:4])
        elif(gameDict['month'] <= 4):
            gameDict['date']+= "/" + str(int(gameDict['season'][:4])+1)
            gameDict['year']=int(gameDict['season'][:4])+1
        gameDict['BUScore'],gameDict['OppoScore']=int(gameDict['scoreline'].split('-')[0]),int(gameDict['scoreline'].split('-')[1])
        gameDict['GD']=abs(gameDict['BUScore']-gameDict['OppoScore'])
        if(gameDict['season']=='1973-74' and gameDict['date']=='12/12/1973'):
            coach='Jack Parker'
        gameDict['date']=pd.Timestamp(gameDict['date'])
        gameDict['dow']=gameDict['date'].weekday()
        gameList.append(gameDict)
f.close()
dfGames=pd.DataFrame(gameList)

In [4]:
def convertToInt(val):
        if(val.isdigit()):
            val=int(val)
        else:
            val=None
        return val

In [5]:
def convertToFloat(val):
        try:
            val=float(val)
        except:
            val=float('nan')
        return val

In [6]:
def convertSeasons(season):
        gap=season.split(',')
        years=season[2:].split('-')
        seasonStr=''
        if(len(gap)>1):
            for i in gap:
                seasonStr+=convertSeasons(i)
        else:
            yearDiff=abs(int(years[0])-int(years[1]))
            if(yearDiff>6):
                yearDiff=100-yearDiff
            firstHalf=season[:4]
            seasonStr=''
            for i in range(yearDiff):
                secondHalf=int(firstHalf)+1
                seasonStr+=str(firstHalf)+'-'+str(secondHalf)[2:]+','
                firstHalf=secondHalf
        return seasonStr[:-1]

In [7]:
def decodeTeam(team):
    origTeam = team
    team=team.lower()
    team=team.replace(" ","")
    team=team.replace("-","")
    team=team.replace("'","")
    team=team.replace(".","")
    dict={"afa" : "Air Force",
        "aic" : "American International",
        "alabamahuntsville" : "Alabama Huntsville",
        "americanintl" : "American International",
        "au" : "Augustana",
        "amworst" : "Massachusetts",
        "amwurst" : "Massachusetts",
        "anosu" : "Ohio State",
        "army" : "Army West Point",
        "asu" : "Arizona State",
        "bama" : "Alabama Huntsville",
        "bc" : "Boston College",
        "bemidji" : "Bemidji State",
        "bgsu" : "Bowling Green",
        "bigred" : "Cornell",
        "bobbymo" : "Robert Morris",
        "boston" : "Boston University",
        "bostonu" : "Boston University",
        "bowlinggreenstate" : "Bowling Green",
        "bruno" : "Brown",
        "bu" : "Boston University",
        "cambridgewarcriminalfactory" : "Harvard",
        "cc" : "Colorado College",
        "cgate" : "Colgate",
        "gate" : "Colgate",
        "chestnuthillcommunitycollege" : "Boston College",
        "chestnuthilluniversity" : "Boston College",
        "clarky" : "Clarkson",
        "cct" : "Clarkson",
        "connecticut" : "UConn",
        "cor" : "Cornell",
        "cuse" : "Syracuse",
        "darty" : "Dartmouth",
        "du" : "Denver",
        "duluth" : "Minnesota Duluth",
        "dutchpeople" : "Union",
        "ferris" : "Ferris State",
        "ferriswheel" : "Ferris State",
        "finghawks" : "North Dakota",
        "goofers" : "Minnesota",
        "hc" : "Holy Cross",
        "hu" : "Harvard",
        "howlinhuskies" : "Northeastern",
        "huntsville" : "Alabama Huntsville",
        "icebus" : "UConn",
        "keggy" : "Dartmouth",
        "lakestate" : "Lake Superior State",
        "lakesuperior" : "Lake Superior State",
        "lowell" : "UMass Lowell",
        "lowelltech" : "UMass Lowell",
        "ulowell" : "Umass Lowell",
        "lssu" : "Lake Superior State",
        "lu" : "Lindenwood",
        "liu" : "Long Island University",
        "mack" : "Merrimack",
        "mankato" : "Minnesota State",
        "mc" : "Merrimack",
        "mich" : "Michigan",
        "meatchicken" : "Michigan",
        "mnsu" : "Minnesota State",
        "mrbee" : "American International",
        "msu" : "Michigan State",
        "mtu" : "Michigan Tech",
        "nd" : "Notre Dame",
        "nebraskaomaha" : "Omaha",
        "neu" : "Northeastern",
        "newtonsundayschool" : "Boston College",
        "newhavenwarcriminalfactory" : "Yale",
        "nmu" : "Northern Michigan",
        "northern" : "Northern Michigan",
        "nodak" : "North Dakota",
        "nu" : "Northeastern",
        "osu" : "Ohio State",
        "pc" : "Providence",
        "pianohuskies" : "Michigan Tech",
        "prinny" : "Princeton",
        "psu" : "Penn State",
        "purplecows" : "Minnesota State",
        "qu" : "Quinnipiac",
        "quinny" : "Quinnipiac",
        "quinni" : "Quinnipiac",
        "rmu" : "Robert Morris",
        "rpi" : "Rensselaer",
        "rit" : "RIT",
        "saintas" : "Saint Anselm",
        "scsu" : "St. Cloud State",
        "shu" : "Sacred Heart",
        "slu" : "St. Lawrence",
        "slushbus" : "UConn",
        "smc" : "Saint Michael's",
        "sparky" : "Arizona State",
        "sparty" : "Michigan State",
        "stanselm" : "Saint Anselm",
        "stcloud" : "St. Cloud State",
        "stmichaels" : "Saint Michael's",
        "stmikes" : "Saint Michael's",
        "stthomas" : "St. Thomas",
        "ust" : "St. Thomas",
        "sootech" : "Lake Superior State",
        "su" : "Syracuse",
        "syracuse" : "Syracuse",
        "toothpaste" : "Colgate",
        "uaa" : "Alaska Anchorage",
        "uaf" : "Air Force",
        "uaf" : "Alaska",
        "uah" : "Alabama Huntsville",
        "uconn" : "UConn",
        "umass" : "Massachusetts",
        "uma" : "Massachusetts",
        "umassamherst" : "Massachusetts",
        "umasslowell" : "UMass Lowell",
        "umd" : "Minnesota Duluth",
        "uml" : "UMass Lowell",
        "umo" : "Maine",
        "umaine" : "Maine",
        "umtc" : "Minnesota",
        "umn" : "Minnesota",
        "und" : "North Dakota",
        "unh" : "New Hampshire",
        "uno" : "Omaha",
        "usma" : "Army West Point",
        "uvm" : "Vermont",
        "uw" : "Wisconsin",
        "wisco" : "Wisconsin",
        "wmu" : "Western Michigan",
        "ziggy" : "Bowling Green",
        "zoomass" : "Massachusetts"}

    if team in dict:
        return dict[team]
    else:
        teamName=''
        teamSplit = origTeam.split(' ')
        for i in range(len(teamSplit)):
            teamName+=teamSplit[i].capitalize()
            if(i<len(teamSplit)-1):
                teamName+=' '
        return teamName


In [8]:
# Get Jerseys
fileName=('JerseyNumbers.txt')
tourneys=[]
teamSet=set()
playerList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    numDict={'number':0,'player':'','season':''}
    for i in rows:
        
        numSearch=re.search("#(.*)",i)
        if numSearch != None:
            number=numSearch.group(1)
        if("Retired - " in i):
                continue
        playerSearch=re.search("(\d*-\d*) (.*)",i)
        if playerSearch != None:
            season=playerSearch.group(1)
            numDict={'number':int(number),
                 'season':convertSeasons(season),
                 'name':playerSearch.group(2)}
            playerList.append(numDict)
f.close()
dfJersey=pd.DataFrame(playerList)

In [9]:
fileName=('SkaterStats.txt')
tourneys=[]
teamSet=set()
skateList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        skaterSearch=re.search('(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)\/(\S*)',i)
        if skaterSearch!=None:
            skaterDict={'last':skaterSearch.group(1),
                       'first':skaterSearch.group(2),
                        'name':skaterSearch.group(2)+' '+skaterSearch.group(1),
                       'seasons':convertSeasons(skaterSearch.group(3)),
                       'gp':convertToInt(skaterSearch.group(4)),
                       'goals':convertToInt(skaterSearch.group(5)),
                       'assists':convertToInt(skaterSearch.group(6)),
                       'pts':convertToInt(skaterSearch.group(7)),
                       'pen':convertToInt(skaterSearch.group(8)),
                       'pim':convertToInt(skaterSearch.group(9))}
            skateList.append(skaterDict)
dfSkate=pd.DataFrame(skateList)

In [10]:
fileName=('GoalieStats.txt')
tourneys=[]
teamSet=set()
goalieList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        goalieSearch=re.search('(.*),(.*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*) (\S*)',i)
        if goalieSearch!=None:
            mins=goalieSearch.group(5).split(':')
            if(len(mins)>1):
                time = "{}:{}".format(*divmod(int(mins[0]), 60)) + ":" + mins[1]
                time = pd.to_timedelta(time)
            else:
                time=float('nan')
            goalieDict={'last':goalieSearch.group(1),
                       'first':goalieSearch.group(2),
                       'name':goalieSearch.group(2)+' '+goalieSearch.group(1),
                       'seasons':convertSeasons(goalieSearch.group(3)),
                       'gp':convertToInt(goalieSearch.group(4)),
                       'mins':round(pd.Timedelta(time).total_seconds()/60,2),
                       'ga':convertToInt(goalieSearch.group(6)),
                       'gaa':convertToFloat(goalieSearch.group(7)),
                       'saves':convertToInt(goalieSearch.group(8)),
                       'sv%':convertToFloat(goalieSearch.group(9)),
                       'W':convertToInt(goalieSearch.group(10)),
                       'L':convertToInt(goalieSearch.group(11)),
                       'T':convertToInt(goalieSearch.group(12))}
            goalieList.append(goalieDict)
dfGoalie=pd.DataFrame(goalieList)            

In [11]:
fileName=('SeasonLeaders.txt')
leadList=[]
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    for i in rows:
        leadSearch=re.search('(\d{4}-\d{2}) (\d*) (.*) (\d*) (.*) (\d*) (.*)',i)
        if leadSearch!=None:
            leadDict={'season':leadSearch.group(1),
                      'year': int(leadSearch.group(1)[:4])+1,
                       'goals':convertToInt(leadSearch.group(2)),
                        'gname':leadSearch.group(3),
                       'assists':convertToInt(leadSearch.group(4)),
                       'aname':leadSearch.group(5),
                       'pts':convertToInt(leadSearch.group(6)),
                       'pname':leadSearch.group(7)}
            leadList.append(leadDict)
f.close()
dfLead=pd.DataFrame(leadList)

In [81]:
fileName=('BeanpotHistory.txt')
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    beanList=[]
    for i in rows:
        col=i.split('\t')
        if(col[3]!=''):
            beanDict={"edition" : col[0],
                    "year" : col[1],
                    "arena" : col[2],
                    "semiDate" : col[3],
                    "semiAttend" : int(col[4]),
                    "semi1Winner" : col[5],
                    "semi1WinnerScore" : int(col[6]),
                    "semi1Loser" : col[7],
                    "semi1LoserScore" : int(col[8]),
                    "semi1OT" : col[9],
                    "semi2Winner" : col[10],
                    "semi2WinnerScore" : col[11],
                    "semi2Loser" : col[12],
                    "semi2LoserScore" : int(col[13]),
                    "semi2OT" : col[14],
                    "champDate" : col[15],
                    "champAttend" : int(col[16]),
                    "consWinner" : col[17],
                    "consWinnerScore" : int(col[18]),
                    "consLoser" : col[19],
                    "consLoserScore" : int(col[20]),
                    "consOT" : col[21],
                    "champion" : col[22],
                    "championScore" : int(col[23]),
                    "runnerup" : col[24],
                    "runnerupScore" : int(col[25]),
                    "champOT" : col[26]}
            beanDict['semiDOW'],beanDict['semiDate']=beanDict['semiDate'].split(',')
            beanDict['champDOW'],beanDict['champDate']=beanDict['champDate'].split(',')
            beanDict['semiDate']=beanDict['semiDate'].rstrip(' ').lstrip(' ')
            beanDict['champDate']=beanDict['champDate'].rstrip(' ').lstrip(' ')
            beanDict['semiDate']+='/'+beanDict['year']
            beanDict['champDate']+='/'+beanDict['year']
            beanDict['semiDate']=pd.Timestamp(beanDict['semiDate'])
            beanDict['champDate']=pd.Timestamp(beanDict['champDate'])
            beanList.append(beanDict)
f.close()
dfBeanpot=pd.DataFrame(beanList)

In [13]:
def determineQueryType(query):
    qType=''
    if('record' in query and 'career' not in query):
        qType='record'
    elif('wins' in query):
        qType='wins'
    elif('loses' in query):
        qType='loses'
    elif('ties' in query):
        qType='ties'
    else:
        qType='player'
    return qType

In [14]:
def tokenizeResultsQuery(query):
    keywords=['vs','at','under','since','after','between','with','before','from','in','on','against']
    keyDict={}
    keyWordsDict={}
    for i in keywords:
        if(query.find(i+' ')>=0):
            finds=[m.start() for m in re.finditer(i+' ', query)]
            for d in range(len(finds)):
                if(finds[d]>0 and query[finds[d]-1]!=' '):
                    finds.pop(d)
            if(finds==[]):
                continue
            elif(len(finds)>1):
                counter=0
                for d in range(len(finds)):
                    if(finds[d]==0):
                        finds[d]=1
                        query=' '+query
                    keyDict[i+'_'+str(counter)]=finds[d]
                    counter+=1
            else:
                if(finds[0]==0):
                    finds[0]=1
                    query=' '+query
                keyDict[i]=finds[0]
    sorted_key = sorted(keyDict.items(), key=operator.itemgetter(1))
    keyWordList=[]
    for i in sorted_key:
        keyWordList.append(i[0])
        keyWordsDict={}
    tokens=re.split(" | ".join(keywords),query)
    if(tokens[0]==' ' or tokens[0]==''):
        tokens.pop(0)
    if(keyWordList!=[]):
        for i in range(len(tokens)):
            tokens[i]=tokens[i].rstrip(' ').lstrip(' ')
            keyWordsDict[keyWordList[i]]=tokens[i]
    return keyWordsDict

In [15]:
def cleanupQuery(query,qType):
    cleanlist=['the','of',"bu","bu's",'what','is',"what's",'number of','games','game','his','arena','rink',"'s", "a", "an"]
    for i in cleanlist:
        query=query.replace(i+' ','')
    if(qType!=''):
        query=query.replace(qType+' ','')
    return query.lower()

In [16]:
def getResults(dfGames,query):
    global tourneyDict
    months=[x.lower() for x in list(calendar.month_name)]
    months_short=[x.lower() for x in list(calendar.month_abbr)]
    days=[x.lower() for x in list(calendar.day_name)]
    days_short=[x.lower() for x in list(calendar.day_abbr)]
    qType=determineQueryType(query)
    queryDict=tokenizeResultsQuery(cleanupQuery(query,qType))

    if (queryDict=={} or qType=='' or qType=='player'):
        return ''
    print(queryDict)
    numGames=''
    ascen=True
    dfQueryList = []
    if('vs' in queryDict.keys()):
        print("Opponent is:",decodeTeam(queryDict['vs']))
        dfQueryList.append("(dfGames['opponent'].str.contains('{}',case=False))".format(decodeTeam(queryDict['vs'])))
    if('against' in queryDict.keys()):
        print("Opponent is:",decodeTeam(queryDict['against']))
        dfQueryList.append("(dfGames['opponent'].str.contains('{}',case=False))".format(decodeTeam(queryDict['against'])))
    if('under' in queryDict.keys()):
        print('Coach is: ', queryDict['under'])
        if(queryDict['under']=='AOC'.lower()):
            queryDict['under']="Albie O'Connell"
        dfQueryList.append("(dfGames['coach'].str.contains(\"{}\",case=False))".format(queryDict['under']))
    if('with' in queryDict.keys()):
        print('Coach is: ', queryDict['with'])
        if(queryDict['with']=='AOC'.lower()):
            queryDict['with']="Albie O'Connell"
        dfQueryList.append("(dfGames['coach'].str.contains(\"{}\",case=False))".format(queryDict['with']))
    if('at' in queryDict.keys()):
        if(queryDict['at'].capitalize() in ['Home','Away','Neutral']):
            print('Location is:',queryDict['at'])
            dfQueryList.append("(dfGames['location']==(\"{}\"))".format(queryDict['at']))
        elif(dfGames.loc[(dfGames['opponent']==decodeTeam(queryDict['at']))]['opponent'].count()>0):
            print('Opponent is:',decodeTeam(queryDict['at']))
            dfQueryList.append("(dfGames['opponent'].str.contains('{}',case=False))".format(decodeTeam(queryDict['at'])))
            print('Location is:','Away')
            dfQueryList.append("(dfGames['location']=='Away')".format(queryDict['at']))
        else:
            print('Location is: ',queryDict['at'])
            dfQueryList.append("(dfGames['arena'].str.contains(\"{}\",case=False))".format(queryDict['at']))
    if('on' in queryDict.keys()):
        dateSearch=re.search('(\w*).(\d*)',queryDict['on'])
        if(dateSearch !=None and '' not in dateSearch.groups()):            
            print('Month: {} Day:{}'.format(dateSearch.group(1),dateSearch.group(2)))
            for i in range(len(months_short)):
                if(months_short[i] in dateSearch.group(1)):                
                    dfQueryList.append("(dfGames['month']=={})".format(int(dateSearch.group(1))))
                    break
            dfQueryList.append("(dfGames['day']=={})".format(int(dateSearch.group(2))))
        else:
            for i in range(len(days_short)):
                if(days_short[i] in queryDict['on']):
                    print('DOW: ',days_short[i])
                    dfQueryList.append("(dfGames['dow']=={})".format(i))
                    break
    if('in' in queryDict.keys()):
        digSearch=re.search('\d',queryDict['in'])
        decSearch=re.search('(\d{2,4})s',queryDict['in'])
        seaSearch=re.search('(\d{4}-\d{2})',queryDict['in'])
        if(digSearch==None and not(queryDict['in'] in months) and not(queryDict['in'] in months_short)):
            print('Tourney is:',queryDict['in'])
            if('NCAA' in queryDict['in'].upper()):
                queryDict['in']=queryDict['in'].replace('s','')
            if(queryDict['in'].upper() in tourneyDict.keys()):
                queryDict['in']=tourneyDict[queryDict['in'].upper()]
            dfQueryList.append("(dfGames['tourney'].str.contains(\"{}\",case=False))".format(queryDict['in']))
        elif(decSearch!=None):
            if(len(decSearch.group(1))==2):
                if(int(decSearch.group(1))>20):
                    decadeStart=int("19"+decSearch.group(1))
                else:
                    decadeStart=int("20"+decSearch.group(1))
            else:
                decadeStart=int(decSearch.group(1))
            dfQueryList.append("(dfGames['date'].between('{}','{}'))".format(decadeStart,decadeStart+9))
        elif(seaSearch!=None):
            print('Season ',seaSearch.group(1))
            dfQueryList.append("(dfGames['season']==\"{}\")".format(queryDict['in']))
        elif('first' in queryDict['in']):
            numGamesSearch=re.search('first (\d*)',queryDict['in'])
            if(numGamesSearch!=None):
                print('First ',numGamesSearch.group(1))
                numGames=int(numGamesSearch.group(1))
        elif('last' in queryDict['in']):
            numGamesSearch=re.search('last (\d*)',queryDict['in'])
            if(numGamesSearch!=None):
                print('last:',numGamesSearch.group(1))
                numGames=int(numGamesSearch.group(1))
                ascen=False
        else:
            print('year: ',queryDict['in'])
            if(queryDict['in'].isdigit()):
                dfQueryList.append("(dfGames['year']=={})".format(queryDict['in']))
            elif(queryDict['in'] in months):                
                dfQueryList.append("(dfGames['month']=={})".format(months.index(queryDict['in'])))
            elif(queryDict['in'] in months_short):
                dfQueryList.append("(dfGames['month']=={})".format(months_short.index(queryDict['in'])))
                                                         
    if('since' in queryDict.keys()):
        print('Since: ',queryDict['since'])
        dfQueryList.append("(dfGames['date']>'{}')".format(queryDict['since']))
            
    if('before' in queryDict.keys()):
        print('before: ',queryDict['before'])
        dfQueryList.append("(dfGames['date']<'{}')".format(queryDict['before']))
    if('after' in queryDict.keys()):
        print('after: ',queryDict['after'])
        dfQueryList.append("(dfGames['date']>'{}')".format(queryDict['after']))
    if('between' in queryDict.keys()):
        print('Between: ',queryDict['between'].split(' and '))
        dates=queryDict['between'].split(' and ')
        dfQueryList.append("(dfGames['date'].between('{}','{}'))".format(dates[0],dates[1]))
    if('from' in queryDict.keys()):
        print('From: ',queryDict['from'].split(' to '))
        dates=queryDict['from'].split(' to ')
        dfQueryList.append("(dfGames['date'].between('{}','{}'))".format(dates[0],dates[1]))
    dfQuery =''
    for i in dfQueryList:
        dfQuery += i + " & "
    dfQuery=dfQuery[:-2]
    result=''
    dfResult=eval("dfGames.loc[{}].sort_values('date',ascending={})[:{}]".format(dfQuery,ascen,numGames))
    print(dfQuery)
    #print(dfResult)
    if(qType=='record'):
        for i in ['W','L','T']:
            if((dfResult['result']==i).any()):
                res=dfResult.groupby('result').count()['date'][i]
            else:
                res=0
            result+=str(res)+'-'
    if(qType=='wins'):
            if((dfResult['result']=='W').any()):
                res=dfResult.groupby('result').count()['date']['W']
            else:
                res=0
            result=str(res)
    elif(qType=='loses'):
            if((dfResult['result']=='L').any()):
                res=dfResult.groupby('result').count()['date']['L']
            else:
                res=0
            result=str(res)
    elif(qType=='ties'):
            if((dfResult['result']=='T').any()):
                res=dfResult.groupby('result').count()['date']['T']
            else:
                res=0
            result=str(res)
    if(result[-1]=='-'):   
        result=result[:-1]
    if(result!=''):
        return result
    else:
        return "No Results Found"

In [52]:
def getPlayerStats(playerDfs,query):
    dfSkate=playerDfs['careerSkaters']
    dfGoalie=playerDfs['careerGoalies']
    dfJersey=playerDfs['jerseys']
    dfLead=playerDfs['seasonleaders']
    dfSeasSkate=playerDfs['seasonSkaters']
    dfSeasGoalie=playerDfs['seasonGoalies']
    query=cleanupQuery(query,'')
    careerSearch=re.search('(.*) career (.*)',query)
    numSearch=re.search('\#(\d*)',query)
    nameSearch=re.search('by (\w*)',query)
    seasonSearch=re.search('in (\d{4}-\d{2}|\d{4})',query)
    if(seasonSearch!=None):
        season=seasonSearch.group(1)
        if('-' in seasonSearch.group(1)):
            year=int(season[:4])+1
        else:
            year=int(season)
    if('most' in query or 'lead' in query):

        if(seasonSearch!=None):
            if(re.search('goal\S',query)):
                statType='goals'
                name='gname'
            elif(re.search('pts|point|scor',query)):
                statType='pts'
                name='pname'
                return "{}:{}-{}--{}".format(dfLead.loc[(dfLead['year']==year)][name].to_string(index=False).lstrip(),dfLead.loc[(dfLead['year']==year)]['goals'].to_string(index=False).lstrip(),dfLead.loc[(dfLead['year']==year)]['assists'].to_string(index=False).lstrip(),dfLead.loc[(dfLead['year']==year)]['pts'].to_string(index=False).lstrip())

            elif(re.search('assist\S',query)):
                statType='assists'
                name='aname'
               
            return "{}:{} {}".format(dfLead.loc[(dfLead['year']==year)][name].to_string(index=False).lstrip(),dfLead.loc[(dfLead['year']==year)][statType].to_string(index=False).lstrip(),statType)
        elif(numSearch!=None):
            number=int(numSearch.group(1))
            dfNum=dfSkate.loc[dfSkate['name'].isin((dfJersey.loc[dfJersey['number']==number]['name']))]
            if(re.search('goal\S',query)):
                statType='goals'
            elif(re.search('pts|point|scor',query)):
                statType='pts'
                df=dfNum.sort_values(statType,ascending=False)[:1]
                name=df['name'].to_string(index=False).lstrip(' ')
                pts=df['pts'].to_string(index=False).lstrip(' ')
                goals=df['goals'].to_string(index=False).lstrip(' ')
                assists=df['assists'].to_string(index=False).lstrip(' ')
                if(not df.empty):
                    return "{}:{}-{}--{}".format(name,goals,assists,pts)
            elif(re.search('assist\S',query)):
                statType='assists'
            df=dfNum.sort_values(statType,ascending=False)[:1]
            if(not df.empty):
                name=df['name'].to_string(index=False).lstrip(' ')
                stat=df[statType].to_string(index=False).lstrip(' ')
                return "{}: {} {}".format(name,stat,statType)
        elif(nameSearch!=None):
            name=nameSearch.group(1)
            dfName=dfSkate.loc[dfSkate['name'].str.contains(name,case=False)]
            if(re.search('goal\S',query)):
                statType='goals'
            elif(re.search('pts|point|scor',query)):
                statType='pts'
                df=dfName.sort_values(statType,ascending=False)[:1]
                name=df['name'].to_string(index=False).lstrip(' ')
                pts=df['pts'].to_string(index=False).lstrip(' ')
                goals=df['goals'].to_string(index=False).lstrip(' ')
                assists=df['assists'].to_string(index=False).lstrip(' ')
                if(not df.empty):
                    return "{}:{}-{}--{}".format(name,goals,assists,pts)
            elif(re.search('assist\S',query)):
                statType='assists'
            df=dfName.sort_values(statType,ascending=False)[:1]
            name=df['name'].to_string(index=False).lstrip(' ')
            stat=df[statType].to_string(index=False).lstrip(' ')
            if(not df.empty):
                return "{}: {} {}".format(name,stat,statType)
        else:
            if(re.search('goal\S',query)):
                statType='goals'
            elif(re.search('pts|point|scor',query)):
                statType='pts'
                df=dfSkate.sort_values(statType,ascending=False)[:1]
                name=df['name'].to_string(index=False).lstrip(' ')
                pts=df['pts'].to_string(index=False).lstrip(' ')
                goals=df['goals'].to_string(index=False).lstrip(' ')
                assists=df['assists'].to_string(index=False).lstrip(' ')
                if(not df.empty):
                    return "{}:{}-{}--{}".format(name,goals,assists,pts)
            elif(re.search('assist\S',query)):
                statType='assists'
            df=dfSkate.sort_values(statType,ascending=False)[:1]
            name=df['name'].to_string(index=False).lstrip(' ')
            stat=df[statType].to_string(index=False).lstrip(' ')
            if(not df.empty):
                return "{}: {} {}".format(name,stat,statType) 
    if(numSearch !=None and seasonSearch != None):
        number=int(numSearch.group(1))
        season=seasonSearch.group(1)
        if(not dfJersey.loc[(dfJersey['number']==number) & (dfJersey['season'].str.contains(season))].empty):
            return dfJersey.loc[(dfJersey['number']==number) & (dfJersey['season'].str.contains(season))]['name'].to_string(index=False).lstrip()
        elif(number==6 and int(season[:4])>=2014):
            return "Retired - Jack Parker"
        elif(number==24 and int(season[:4])>=1999):
            return "Retired - Travis Roy"
        else:
            return "No one"
    if(careerSearch!=None):
        playerName=careerSearch.group(1)
        stat=careerSearch.group(2)
        pStatsLine=dfSkate.loc[dfSkate['name'].str.contains(playerName,case=False)]
        gStatsLine=dfGoalie.loc[dfGoalie['name'].str.contains(playerName,case=False)]
        if(len(pStatsLine)==1):
            goals=pStatsLine['goals'].to_string(index=False).lstrip()
            assists=pStatsLine['assists'].to_string(index=False).lstrip()
            pts=pStatsLine['pts'].to_string(index=False).lstrip()
            if('stats' in stat or 'stat line' in stat.replace(' ','')):
                return("{}-{}--{}".format(goals,assists,pts))
            elif('goal' in stat):
                return goals
            elif('assist' in stat):
                return assists
            elif('pts' in stat or 'point' in stat):
                return pts
        elif(len(gStatsLine)==1):
            gaa=gStatsLine['gaa'].to_string(index=False).lstrip()
            svper=gStatsLine['sv%'].to_string(index=False).lstrip()
            wins=int(float(gStatsLine['W'].to_string(index=False).lstrip()))
            loss=int(float(gStatsLine['L'].to_string(index=False).lstrip()))
            tie=int(float(gStatsLine['T'].to_string(index=False).lstrip()))
            if('stats' in stat or 'stat line' in stat.replace(' ','')):
                return("{}/{}/{}-{}-{}".format(gaa,svper,wins,loss,tie))
            elif('gaa' in stat):
                return gaa
            elif('sv' in stat or 'save' in stat):
                return svper
            elif('record' in stat):
                return "{}-{}-{}".format(wins,loss,tie)
            
    return ''
    

In [95]:
def getBeanpotStats(dfBeanpot,query):
    beanNumSearch=re.search('(\d{4}|\d{1,2}|first|second|third)(?:st|nd|rd|th)? beanpot',query)
    typeSearch=re.search('beanpot (champion|winner|runner|runner-up|1st|first|2nd|second|third|3rd|fourth|4th|last|results)',query)
    if(typeSearch!=None and beanNumSearch != None):
        qType=typeSearch.group(1)
        year=beanNumSearch.group(1)
        if(year=='2021'):
            return '2021 Beanpot cancelled due to the COVID-19 pandemic'
        if(len(beanNumSearch.group(1))==4):
            dfRes=dfBeanpot.loc[dfBeanpot['year']==year]
        elif(len(beanNumSearch.group(1))<=2):
            dfRes=dfBeanpot.loc[dfBeanpot['edition']==year] 
        elif(year=='first'):
            dfRes=dfBeanpot.loc[dfBeanpot['edition']=='1']
        elif(year=='second'):
            dfRes=dfBeanpot.loc[dfBeanpot['edition']=='2']
        elif(year=='third'):
            dfRes=dfBeanpot.loc[dfBeanpot['edition']=='3']
        if(qType in ['champion','winner','1st','first']):
            return dfRes['champion'].to_string(index=False).lstrip(' ')
        if(qType in ['runner','2nd','second']):
            return dfRes['runnerup'].to_string(index=False).lstrip(' ')
        if(qType in ['third','3rd']):
            return dfRes['consWinner'].to_string(index=False).lstrip(' ')
        if(qType in ['fourth','4th','last']):
            return dfRes['consLoser'].to_string(index=False).lstrip(' ')

In [99]:
getBeanpotStats(dfBeanpot,'34 beanpot winner')

'Boston University'

In [94]:
dfBeanpot.loc[dfBeanpot['year']=='2021']

,edition,year,arena,semiDate,semiAttend,semi1Winner,semi1WinnerScore,semi1Loser,semi1LoserScore,semi1OT,...,consLoser,consLoserScore,consOT,champion,championScore,runnerup,runnerupScore,champOT,semiDOW,champDOW


In [53]:
query='record at matthews in march'
query='most pts in 2010-11'
if(determineQueryType(query)!='player'):
    print(getResults(dfGames,query))  
else:
    playerDfs={}
    playerDfs['careerSkaters']=dfSkate
    playerDfs['careerGoalies']=dfGoalie
    playerDfs['jerseys']=dfJersey
    playerDfs['seasonleaders']=dfLead
    playerDfs['seasonSkaters']=dfSeasSkate
    playerDfs['seasonGoalies']=dfSeasGoalie
    print(getPlayerStats(playerDfs,query))


Alex Chiasson:15-21--34


In [19]:
 dfJersey.loc[(dfJersey['number']==4) & (dfJersey['season'].str.contains('2021-22'))]

,number,season,name
140,4,"2021-22,2022-23",Ty Gallagher


In [20]:
dfNum=dfSkate.loc[dfSkate['name'].isin((dfJersey.loc[dfJersey['number']==3]['name']))]
dfNum.sort_values('pts',ascending=False)[:1]['name'].to_string(index=False).lstrip(' ')

'Vic Stanfield'

In [21]:
x='this is a test'
pat='bo'
re.search(pat,x)

In [22]:
x=dfSkate.loc[(dfSkate['name']).str.contains('Jay Pandolfo')].sort_values('pts',ascending=False)

In [23]:
dfSkate.loc[(dfSkate['name']).str.contains('Green')].sort_values('pts',ascending=False)

,last,first,name,seasons,gp,goals,assists,pts,pen,pim
292,Greenway,Jordan,Jordan Greenway,"2015-16,2016-17,2017-18",112.0,28,64,92,84.0,192.0
291,Green,Richard,Richard Green,"1962-63,1963-64",39.0,6,24,30,NaN,62.0


In [24]:
dfGames.loc[(dfGames['opponent'].str.contains('Boston College',case=False)) & (dfGames['month']==2)  ].sort_values('date',ascending=False)[:2]

,date,result,ot,arena,opponent,gameType,tourney,scoreline,location,coach,...,conference,season,note,month,day,year,BUScore,OppoScore,GD,dow
2794,2022-02-27,L,None,Conte Forum,Boston College,Conference,None,1-3,Away,Albie O'Connell,...,Hockey East,2021-22,,2,27,2022,1,3,2,6
2793,2022-02-26,W,None,Agganis Arena,Boston College,Conference,None,6-3,Home,Albie O'Connell,...,Hockey East,2021-22,,2,26,2022,6,3,3,5


In [25]:
qWords=query.split(' ')
for i in qWords:
    team=decodeTeam('')
    if(dfGames.loc[(dfGames['opponent']==decodeTeam(i))]['opponent'].count()>0):
        print(i)

In [28]:
dfBeanpot

,edition,year,arena,semiDate,semiAttend,semi1Winner,semi1WinnerScore,semi1Loser,semi1LoserScore,semi1OT,...,consLoser,consLoserScore,consOT,champion,championScore,runnerup,runnerupScore,champOT,semiDOW,champDOW
0,1,1952,Boston Arena,1952-12-26,5105,Boston University,4,Northeastern,1,,...,Northeastern,0,,Harvard,7,Boston University,4,,Fri,Sat
1,2,1954,Boston Garden,1954-01-11,711,Harvard,3,Boston University,2,,...,Northeastern,3,,Boston College,4,Harvard,1,,Mon,Tue
2,3,1955,Boston Garden,1955-02-07,2560,Harvard,12,Northeastern,3,,...,Northeastern,3,,Harvard,5,Boston College,4,OT,Mon,Tue
3,4,1956,Boston Garden,1956-02-06,2500,Boston College,7,Northeastern,1,,...,Northeastern,3,,Boston College,4,Harvard,2,,Mon,Wed
4,5,1957,Boston Garden,1957-02-01,4038,Boston College,6,Northeastern,0,,...,Northeastern,3,,Boston College,5,Boston University,4,OT,Fri,Tue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,2017,TD Garden,2017-02-06,15299,Harvard,4,Northeastern,3,,...,Boston College,2,,Harvard,6,Boston University,3,,Mon,Mon
65,66,2018,TD Garden,2018-02-05,13567,Northeastern,3,Boston College,0,,...,Boston College,4,OT,Northeastern,5,Boston University,2,,Mon,Mon
66,67,2019,TD Garden,2019-02-04,12413,Boston College,2,Harvard,1,,...,Boston University,2,,Northeastern,4,Boston College,2,,Mon,Mon
67,68,2020,TD Garden,2020-02-03,13141,Northeastern,3,Harvard,1,,...,Harvard,2,,Northeastern,5,Boston University,4,2OT,Mon,Mon


In [29]:
### Current Team ###

In [30]:
currSeason='2022-23'
url = "https://goterriers.com/sports/mens-ice-hockey/stats?path=mhockey"
#print(url)
f=urllib.request.urlopen(url)
html = f.read()
f.close()
soup = BeautifulSoup(html, 'html.parser')
head=soup.find('h1')
if(currSeason not in head):
    print('Old')
else:
    curSkate=soup.find('section',{'id':'individual-overall-skaters'})
    rows=curSkate.find_all('tr')
    currSkaters=[]
    for i in rows:
        col=i.find_all('td')
        name=i.find('span')

        if(name!=None):
            counter=0
            lastName,firstName=name.get_text().split(', ')
            skateDict={'number':int(col[0].get_text()),
                       'last':lastName,
                       'first':firstName,
                       'name':firstName+' '+lastName,
                        'gp':int(col[1].get_text()),
                        'goals':int(col[2].get_text()),
                        'assists':int(col[3].get_text()),
                        'pts':int(col[4].get_text()),
                        'pen':col[16].get_text().replace('-','/')}
            currSkaters.append(skateDict)
    dfCurr=pd.DataFrame(currSkaters)



    curGoals=soup.find('section',{'id':'individual-overall-goaltenders'})
    rows=curGoals.find_all('tr')
    currGoalies=[]
    for i in rows:
        col=i.find_all('td')
        name=i.find('span')

        if(name!=None):
            lastName,firstName=name.get_text().split(',')
            goalDict={'number':int(col[0].get_text()),
                       'last':lastName,
                       'first':firstName,
                       'name':firstName+' '+lastName,
                        'gp':int(col[1].get_text().split('-')[0]),
                        'mins':col[2].get_text(),
                        'ga':col[3].get_text(),
                        'gaa':col[4].get_text(),
                        'saves':col[8].get_text(),
                        'sv%':col[9].get_text(),
                        'W':col[10].get_text(),
                        'L':col[11].get_text(),
                        'T':col[12].get_text(),
                        'SO':col[13].get_text()}
            currGoalies.append(goalDict)

    curGoaldf=pd.DataFrame(currGoalies)

Old


In [32]:
curGoaldf

NameError: name 'curGoaldf' is not defined

In [33]:
fileName=('SeasonSkaterStats.txt')
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    seasList=[]
    for i in rows:
        col=i.split('\t')
        seasDict={'number':col[0],
                 'name':col[1],
                 'pos':col[2],
                 'yr':col[3],
                 'gp':int(col[4]),
                 'goals':int(col[5]),
                 'assists':int(col[6]),
                 'pts':int(col[7]),
                 'pens':col[8],
                 'season':col[9],
                 'year':int(col[9][:4])+1}
        seasList.append(seasDict)
f.close()
dfSeasSkate=pd.DataFrame(seasList)

In [34]:
fileName=('SeasonGoalieStats.txt')
with open(fileName, 'r', encoding='utf-8') as f:
    read_data = f.read()
    rows=read_data.split('\n')
    number=0
    seasGList=[]
    for i in rows:
        col=i.split('\t')
        seasGDict={'number':col[0],
                 'name':col[1],
                 'yr':col[2],
                 'gp':int(col[3]),
                 'mins':col[4],
                 'ga':int(col[5]),
                 'saves':int(col[6]),
                 'sv%':float(col[7]),
                 'gaa':float(col[8]),
                 'record':col[9],
                 'SO':int(col[10]),
                 'season':col[11],
                 'year':int(col[11][:4])+1}
        seasGList.append(seasGDict)
f.close()
dfSeasGoalie=pd.DataFrame(seasGList)

In [43]:
x=dfSeasSkate.groupby('name').sum()
x.loc[x['pts'].between(75,100)].sort_values('pts',ascending=False)

,gp,goals,assists,pts,year
name,,,,,
Alex Chiasson,108,36,63,99,6033
Jason Lawrence,158,46,51,97,8030
Matt Grzelcyk,125,26,69,95,8058
David Van der Gulik,141,52,41,93,8018
Cason Hohmann,145,32,60,92,8054
Jordan Greenway,112,28,64,92,6051
Matt Gilroy,160,25,67,92,8030
Colin Wilson,80,29,61,90,4017
Brad Zancanaro,161,34,56,90,8018
